In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-147874
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-147874


In [2]:
# Create compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

# Create a custom environment that encapsulates the training script's dependencies
# (1) Define your conda dependencies in a YAML file
# (2) Create an Azure ML environment from this Conda environment specification.

Creating........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.8.1
- pip=20.1.1
- pip:
    - azureml-defaults
    - scikit-learn

Writing conda_dependencies.yml


In [4]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [12]:
from azureml.widgets import RunDetails
# from azureml.train.sklearn import SKLearn >> DEPRECATED. Use the ScriptRunConfig object with your own defined environment.
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
    'C': uniform(0.01, 100.0), # For regularization
    'max_iter': choice(500, 1000, 1500, 2000, 2500) # Max number of epochs
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.makedirs("training", exist_ok=True)

# !Create a SKLearn estimator is DEPRECATED > Use:
# Create a ScriptRunConfig object to specify the configuration details of your training job

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    max_duration_minutes=30)



In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7
Web View: https://ml.azure.com/runs/HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-147874/workspaces/quick-starts-ws-147874&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-06-24T04:56:36.137029][API][INFO]Experiment created<END>\n""<START>[2021-06-24T04:56:36.677931][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-06-24T04:56:36.886385][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7
Web View: https://ml.azure.com/runs/HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-147874/workspaces/quick-starts-ws-147874&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-06-24T04:56:35.859579Z',
 'endTimeUtc': '2021-06-24T05:14:16.777209Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'fc4c598f-588e-4170-8ffe-8047f6e2e7bf',
  'score': '0.8982762806506434',
  'best_child_run_id': 'HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7_6',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg147874.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7

In [14]:
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('Regularization Strength:', best_run_metrics['Regularization Strength:'])
print('Max iterations:', best_run_metrics['Max iterations:'])
print('Accuracy:', best_run_metrics['Accuracy'])

with open('best_hdr_run_metrics.txt', 'w') as file:
    file.write(str(best_run_metrics))
with open('best_hdr_run_details.txt', 'w') as file:
    file.write(str(best_run.get_details()))

Best Run Id:  HD_df5e532c-42a0-4fc1-ae6b-c3df734fc2b7_6
Regularization Strength: 47.00488472299381
Max iterations: 1000
Accuracy: 0.8982762806506434


In [15]:
# Check the path to the model
best_run.get_file_names()[-1]

'outputs/model.joblib'

In [16]:
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

In [17]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

#model = run.register_model(model_name='sklearn-logistic-regression', model_path="./outputs/model.joblib") # causes ModelPathNotFoundException
model = Model.register(ws, model_path='outputs/model.joblib', model_name='sklearn-logistic-regression', tags=best_run_metrics)


Registering model sklearn-logistic-regression


In [18]:
print(model.name, model.id, model.version, sep='\t')

sklearn-logistic-regression	sklearn-logistic-regression:1	1


In [39]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from train import get_X_y

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = pd.read_csv(url_path)

# Transform data in line with EDA
ds = get_X_y(ds, context='aml')

# Store dataset in the CSV format
os.makedirs('./data', exist_ok=True)
with open('./data/data.csv', 'w') as writer:
    ds.to_csv(writer, index=False)

# Upload data to AzureBlobDatastore
blob_store = ws.get_default_datastore()
blob_store.upload(src_dir='data', target_path='data', overwrite=True)

# Create final dataset for AutoML
training_data = TabularDatasetFactory.from_delimited_files(blob_store.path('data/data.csv'))

Uploading an estimated of 3 files
Uploading data/.amlignore
Uploaded data/.amlignore, 1 files out of an estimated total of 3
Uploading data/.amlignore.amltmp
Uploaded data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading data/data.csv
Uploaded data/data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
automl_settings = {
    'experiment_timeout_minutes' : 30,
    'n_cross_validations' : 3,
    'enable_early_stopping' : True,
    'iteration_timeout_minutes' : 5,
    'max_concurrent_iterations' : 4,
    'max_cores_per_iteration' : -1
}

automl_config = AutoMLConfig(
    task='classification',
    primary_metric='AUC_weighted',
    compute_target=cpu_cluster,
    training_data=training_data,
    label_column_name='y',
    **automl_settings)

In [41]:
# Submit your automl run
expaml = Experiment(workspace=ws, name="udacity-project-automl")
runaml = expaml.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-automl,AutoML_c63d57d1-1726-440d-b0f9-8d2cca8ced58,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [42]:
# Retrieve and save your best automl model.
best_aml_run = runaml.get_best_child()
best_aml_run_metrics = best_aml_run.get_metrics()

print('Best Run Id: ', best_aml_run.id)

best_aml_run.get_file_names()[-4]

Best Run Id:  AutoML_c63d57d1-1726-440d-b0f9-8d2cca8ced58_37


'outputs/model.pkl'

In [43]:
best_aml_run.download_file(best_aml_run.get_file_names()[-4], output_file_path='./outputs/')
aml_model = Model.register(ws, model_path='outputs/model.pkl', model_name='best-aml-model', tags=best_aml_run_metrics)

print(aml_model.name, aml_model.id, aml_model.version, sep='\t')

Registering model best-aml-model
best-aml-model	best-aml-model:2	2


In [44]:
with open('best_aml_run_metrics.txt', 'w') as file:
    file.write(str(best_aml_run_metrics))
with open('best_aml_run_details.txt', 'w') as file:
    file.write(str(best_aml_run.get_details()))

In [45]:
# Delete() is used to deprovision and delete the AmlCompute target. 
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

